###### Last preprocessing step. Removes all non-relevant articles from our corpus. Removes non-relevant information from arxiv-full-database.
###### Adds word count for each context to the data frame
###### Saves the result in database-files

In [41]:
import pandas as pd
import time
import nltk

In [42]:
def flatten(list):
    return "".join([item for sublist in list for item in sublist])
path = '/Users/Svesketerning/Google-Drev/experiments/database-files'
timestr = time.strftime("%Y%m%d-%H%M")

In [43]:
df_arxiv = pd.read_pickle(r'arxiv-full-database-20210123-0907.pickle') 
keep = ['title','abstract','authors_parsed','numpages',
        'numauthors','categories_parsed','numcats','maincat',
       'nummathcats','mainmathcat']
df_arxiv = df_arxiv[keep] # Keep only relevant columns
df_arxiv = df_arxiv.drop(df_arxiv[df_arxiv.maincat != df_arxiv.mainmathcat].index
                         & df_arxiv[df_arxiv.maincat != 'cs.IT'].index
                         & df_arxiv[df_arxiv.maincat != 'math-ph'].index
                        ) # Remove articles without mainmathcat=maincat and cs.IT/math-ph
df_contexts = pd.read_feather('arxiv-contexts.feather')
contexts = [x for x in df_contexts.columns if x!='arxivid']
for c in contexts:
    df_contexts[c] = df_contexts[c].apply(flatten)
df_contexts = df_contexts.set_index('arxivid')
df_contexts.index.names = ['id'] # ArXiv id as index 

article_list = df_arxiv.index.tolist()
df_contexts = df_contexts[df_contexts.index.isin(article_list)] # Drop rows if index not in df_arxiv
# Outcome is all articles with maincat a mathematics category alongside contexts

In [44]:
# word count for each context to the data frame for each article
contextwordcount_df = [] # Counting words in each context per article
for index in range(len(df_contexts.index)):
    counts_article = []
    for c in contexts:
        count = len(nltk.word_tokenize(df_contexts[c][index]))
        counts_article.append(count)
    contextwordcount_df.append({'id':df_contexts.index[index], # Non functional as I list each context
                                'outer': counts_article[0], 'theorem': counts_article[1], 
                                'meta': counts_article[2], 'proof': counts_article[3], 
                                'other': counts_article[4]})
contextwordcount_df = pd.concat([pd.DataFrame(contextwordcount_df[i], index=['id']) for i in range(len(contextwordcount_df))]) 
contextwordcount_df = contextwordcount_df.set_index('id')
# merge to df_arxiv
df_arxiv = df_arxiv.merge(contextwordcount_df, left_index=True, right_index=True)

In [46]:
df_arxiv = df_arxiv.reset_index()
df_contexts = df_contexts.reset_index()
df_contexts.to_feather(path+'/arxiv_contexts'+timestr+'.feather')
df_arxiv.to_feather(path+'/arxiv_extended_database'+timestr+'.feather')